# Problemstellung 

Wenn es zu folgendem Problem kommt, können Sie das Skript unten ausführen, dass alle notwenigen init-Files generiert.


![ModuleNotFoundError.png](/assets/ModuleNotFoundError.png)

# Robustes, selbstheilendes Setup für Jupyter (Erklärung)

Dieses Dokument erklärt die gezeigte Setup‑Zelle **Schritt für Schritt**. Ziel: Dein Notebook findet zuverlässig dein lokales Paket `graphfw` (inkl. `io/writers/xml_writer.py`), auch wenn das Working‑Directory variiert, `__init__.py` fehlt oder der Import‑Cache „hängt“.

---

## TL;DR

* **Paket finden:** gehe vom aktuellen Ordner aus (ggf. `notebooks/` → eine Ebene hoch) und suche nach einem Ordner `graphfw/`.
* **Pakete markieren:** lege fehlende `__init__.py` an, damit Python die Ordner als Pakete erkennt.
* **Pfad priorisieren:** füge den Repo‑Root **vorn** in `sys.path` ein (so hat dein Code Vorrang vor evtl. global installierten Paketen).
* **Cache leeren:** entferne geladene `graphfw`‑Module aus `sys.modules`, damit Änderungen sofort wirken.
* **Import prüfen:** importiere das Modul, zeige die Dateiquelle an und gib bei Fehlern einen klaren Traceback aus.
* **Optional:** Drucke die `pandas`‑Version zur schnellen Umgebungskontrolle.

---

## Referenzcode

```python
# ✅ Robustes, selbstheilendes Setup für Jupyter (graphfw/io/writers/xml_writer)
from __future__ import annotations
from pathlib import Path
import os, sys, importlib, inspect

def find_repo_root(start: Path, marker: str = "graphfw") -> Path:
    p = start.resolve()
    for _ in range(5):  # bis zu 5 Ebenen hoch
        if (p / marker).exists():
            return p
        p = p.parent
    return start.resolve()

def ensure_inits(repo_root: Path) -> list[str]:
    """legt fehlende __init__.py an und gibt die erzeugten Pfade zurück"""
    wanted = [
        repo_root / "graphfw" / "__init__.py",
        repo_root / "graphfw" / "io" / "__init__.py",
        repo_root / "graphfw" / "io" / "writers" / "__init__.py",
    ]
    created = []
    for f in wanted:
        if not f.exists():
            f.parent.mkdir(parents=True, exist_ok=True)
            f.write_text('# package init\n', encoding='utf-8')
            created.append(str(f))
    return created

def purge_modules(prefix: str = "graphfw"):
    """Entlädt geladene Teilmodule aus sys.modules (Import-Cache leeren)."""
    to_del = [k for k in list(sys.modules.keys()) if k == prefix or k.startswith(prefix + ".")]
    for k in to_del:
        del sys.modules[k]

# 1) Repo-Root ermitteln (suche Ordner, der 'graphfw/' enthält)
start = Path.cwd()
if start.name.lower() == "notebooks":
    # häufige Struktur: repo/notebooks/*.ipynb
    start = start.parent
repo_root = find_repo_root(start)

# 2) __init__.py sicherstellen
created = ensure_inits(repo_root)

# 3) sys.path vorn eintragen (Repo-Root soll Vorrang haben)
sys.path.insert(0, str(repo_root))

# 4) Import-Cache für graphfw leeren (wichtig nach Strukturänderungen)
purge_modules("graphfw")

# 5) Sanity-Check & Import
print("CWD:        ", os.getcwd())
print("Repo-Root:  ", repo_root)
print("Neu angelegt:", created if created else "—")

required = [
    repo_root / "graphfw" / "io" / "writers" / "xml_writer.py",
]
missing = [str(p) for p in required if not p.exists()]
print("Fehlend:    ", missing if missing else "—")

# 6) Import & Herkunft anzeigen
try:
    mod = importlib.import_module("graphfw.io.writers.xml_writer")
    print("Geladen aus:", inspect.getfile(mod))
    from graphfw.io.writers.xml_writer import build_xml_path, write_xml
    print("Import OK ✅")
except Exception as e:
    print("Import FEHLER ❌:", repr(e))
    raise

# 7) (Optional) pandas-Version zeigen
import pandas as pd
print("pandas:", pd.__version__)
```

---

## Detaillierte Erklärung

### Prelude & Imports

* `from __future__ import annotations` – **verzögerte** Auswertung von Typannotationen (schneller, forward‑refs einfacher).
* `from pathlib import Path` – objektorientiertes, plattformunabhängiges Pfad‑API.
* `os`, `sys` – Betriebssystemfunktionen, u. a. `os.getcwd()` und `sys.path`.
* `importlib` – dynamische Modulimporte (hier zur Herkunftsanzeige und Testladung).
* `inspect` – Metadaten zum Modul, z. B. physischer Dateipfad via `inspect.getfile(mod)`.

### `find_repo_root(start, marker="graphfw")`

**Zweck:** Finde den **Repo‑Root**, also den Ordner, der den Unterordner `graphfw/` enthält.

* Startpunkt ist das aktuelle Working‑Directory; wenn das Notebook in `notebooks/` liegt, wird automatisch eine Ebene hochgegangen.
* Bis zu **5 Ebenen** nach oben werden durchsucht; bei Treffer wird der Pfad zurückgegeben.

### `ensure_inits(repo_root)`

**Zweck:** Stelle sicher, dass `graphfw/`, `graphfw/io/` und `graphfw/io/writers/` **klassische Pakete** sind.

* Legt fehlende `__init__.py` an (Inhalt: `# package init`).
* Gibt die Liste der neu erzeugten Dateien zurück (für die Statusausgabe).
* Ohne `__init__.py` verhalten sich Ordner evtl. nicht wie Pakete → `import graphfw...` kann scheitern.

### `purge_modules(prefix="graphfw")`

**Zweck:** **Import‑Cache leeren** für alle bereits geladenen `graphfw`‑Teilmodule.

* Lang laufende Jupyter‑Kernel halten alte Modulversionen im Speicher.
* Durch Löschen aus `sys.modules` erzwingen wir, dass beim nächsten Import frisch von der Platte geladen wird.

### Schritt 1: Repo‑Root ermitteln

* `start = Path.cwd()` nimmt das aktuelle Arbeitsverzeichnis.
* Wenn `notebooks/`, dann `start = start.parent` (eine Ebene hoch).
* `repo_root = find_repo_root(start)` sucht den Ordner, der `graphfw/` enthält.

### Schritt 2: `__init__.py` sicherstellen

* `created = ensure_inits(repo_root)` legt fehlende `__init__.py` an und protokolliert neu erzeugte Dateien.

### Schritt 3: `sys.path` vorn eintragen

* `sys.path.insert(0, str(repo_root))` gibt deinem Repo‑Code **Vorrang** gegenüber global installierten Paketen mit gleichem Namen.

### Schritt 4: Import‑Cache leeren

* `purge_modules("graphfw")` räumt alte Einträge auf; wichtig nach Datei‑Umbenennungen/Refactors.

### Schritt 5: Sanity‑Check & Datei‑Existenz

* Druckt **CWD**, **Repo‑Root** und neu angelegte `__init__.py`.
* Prüft, ob die erwartete Datei `graphfw/io/writers/xml_writer.py` existiert.

### Schritt 6: Import & Herkunft anzeigen

* `importlib.import_module("graphfw.io.writers.xml_writer")` lädt das Modul.
* `inspect.getfile(mod)` zeigt die **physische Quelle** (Pfad) des Moduls – wichtig, wenn mehrere Kopien existieren.
* Anschließend Import der Symbole (`build_xml_path`, `write_xml`) und Erfolgsmeldung.
* Bei Fehlern: prägnante Ausgabe und **erneuter Raise** für vollständigen Traceback.

### Schritt 7: Umgebungsschnelltest

* `print("pandas:", pd.__version__)` zeigt die laufende pandas‑Version (z. B. relevant für `to_xml`‑Features).

---

## Häufige Stolpersteine & Tipps

* **Notebook in `notebooks/` gestartet?** Das Skript geht automatisch eine Ebene hoch. Sonst manuell `sys.path.insert(0, "..")`.
* **Fehlende `__init__.py`**: Ohne diese Dateien sind Ordner evtl. keine Pakete → `ensure_inits` fixt das.
* **Alter Import‑Cache**: `purge_modules` erzwingt frische Imports.
* **Namenskonflikte**: Keine Datei `graphfw.py` im Repo‑Root ablegen – würde `graphfw/` überschreiben.
* **Mehrere Repo‑Kopien**: Mit `inspect.getfile(mod)` sicherstellen, dass die **richtige** Datei importiert wurde.

---

## Mini‑Smoke‑Test

```python
import pandas as pd

df = pd.DataFrame({"Id":[1,2], "Title":["A","B"]})
p = write_xml(df, prefix="Test", postfix="Demo", timestamp=False, overwrite=True)
print("Datei:", p, "existiert:", p.exists())
```

Erwartung: Ausgabe bestätigt den Pfad und `existiert: True` – damit ist das Setup betriebsbereit.


In [1]:
# ✅ Robustes, selbstheilendes Setup für Jupyter – rekursiv __init__.py anlegen (graphfw/**)
from __future__ import annotations
from pathlib import Path
import os, sys, importlib, inspect

# ------------------------- Hilfsfunktionen -------------------------

def find_repo_root(start: Path, marker: str = "graphfw") -> Path:
    """
    Steigt bis zu 5 Ebenen nach oben und liefert den ersten Ordner,
    der einen Unterordner 'marker/' (default: 'graphfw') enthält.
    """
    p = start.resolve()
    for _ in range(5):
        if (p / marker).exists():
            return p
        p = p.parent
    return start.resolve()

def ensure_inits_recursive(repo_root: Path, package_dirname: str = "graphfw") -> list[str]:
    """
    Legt rekursiv __init__.py in ALLEN Unterordnern unterhalb von <repo_root>/<package_dirname> an,
    sofern noch nicht vorhanden. Bestehende __init__.py werden NICHT überschrieben.

    Returns:
        Liste der neu angelegten Dateipfade (als Strings).
    """
    gf_dir = (repo_root / package_dirname).resolve()
    if not gf_dir.exists():
        return []

    # Ordner, die wir auslassen (Cache/Hidden/Build/VC)
    skip_names = {
        "__pycache__", ".git", ".hg", ".svn", ".mypy_cache", ".pytest_cache",
        ".ipynb_checkpoints", ".venv", "venv", "env", "build", "dist", ".idea", ".vscode"
    }

    created: list[str] = []

    # Alle Unterordner inkl. Wurzelordner selbst
    dirs = [gf_dir] + [p for p in gf_dir.rglob("*") if p.is_dir()]

    for d in dirs:
        name = d.name.lower()
        if name in skip_names or name.startswith("."):
            continue
        init_file = d / "__init__.py"
        if not init_file.exists():
            d.mkdir(parents=True, exist_ok=True)
            init_file.write_text("# package init\n", encoding="utf-8")
            created.append(str(init_file))

    return created

def purge_modules(prefix: str = "graphfw"):
    """
    Entlädt geladene Teilmodule aus sys.modules (Import-Cache leeren),
    damit Struktur-/Codeänderungen sofort wirken.
    """
    to_del = [k for k in list(sys.modules.keys()) if k == prefix or k.startswith(prefix + ".")]
    for k in to_del:
        del sys.modules[k]

# ------------------------- Setup-Schritte -------------------------

# 1) Repo-Root ermitteln (Ordner, der 'graphfw/' enthält)
start = Path.cwd()
if start.name.lower() == "notebooks":  # typische Struktur: <repo>/notebooks/*.ipynb
    start = start.parent
repo_root = find_repo_root(start, marker="graphfw")

# 2) __init__.py rekursiv sicherstellen
created = ensure_inits_recursive(repo_root, package_dirname="graphfw")

# 3) sys.path vorn eintragen (Repo-Root soll Vorrang haben)
sys.path.insert(0, str(repo_root))

# 4) Import-Cache für graphfw leeren (wichtig nach Strukturänderungen)
purge_modules("graphfw")

# 5) Sanity-Check & Import
print("CWD:         ", os.getcwd())
print("Repo-Root:   ", repo_root)
print("Neu angelegt:", len(created), "Datei(en)")
if created:
    # Nur die ersten paar anzeigen, damit die Ausgabe nicht ausufert
    for p in created[:10]:
        print("  +", p)
    if len(created) > 10:
        print(f"  ... (+{len(created)-10} weitere)")

required = [repo_root / "graphfw" / "io" / "writers" / "xml_writer.py"]
missing = [str(p) for p in required if not p.exists()]
print("Fehlend:     ", missing if missing else "—")

# 6) Import & Herkunft anzeigen
try:
    mod = importlib.import_module("graphfw.io.writers.xml_writer")
    print("Geladen aus: ", inspect.getfile(mod))
    from graphfw.io.writers.xml_writer import build_xml_path, write_xml
    print("Import OK ✅")
except Exception as e:
    print("Import FEHLER ❌:", repr(e))
    raise

# 7) (Optional) pandas-Version zeigen
import pandas as pd
print("pandas:", pd.__version__)


CWD:          c:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw
Repo-Root:    C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API
Neu angelegt: 5 Datei(en)
  + C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw\domains\__init__.py
  + C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw\params\__init__.py
  + C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw\domains\sharepoint\__init__.py
  + C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw\domains\teams\__init__.py
  + C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw\domains\sharepoint\lists\__init__.py
Fehlend:      —
Geladen aus:  C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\graphfw\io\writers\xml_writer.py
Import OK ✅
pandas: 2.3.2
